In [ ]:
import random
import numpy as np
import os
import collections
import pandas as pd
import time

In [ ]:
class ApproximateCounter:
    """
    Counts the frequency of every letter in a given text literary document.
    Metrics such as min, max and average number of occurences are also reported.
    Average pecentage of letter occurence in the text is also reported.
    """

    def __init__(self):
        self.student_counter = 106382
        random.seed = self.student_counter
        self.textFilesDirectory = os.path.join(os.getcwd(), 'TextFiles')
        
        self.counter = 0
        self.prob = 1/2 # probability of the event happening

        self.text_data = None
        self.register = None
        self.count_method = None


    def LoadTextFile(self, filename='sample'):
        self.filename = filename
        filename = filename + '.txt'
        text_file_path = os.path.join(self.textFilesDirectory, 'shakespeare', filename)
        with open(text_file_path, 'r', encoding="utf8") as file:
            text_data = file.readlines()

        # combine all lines from text into a big string
        self.text_data = ''.join(text_data)
        
        # remove all non-alphanumerics characters, and converts to upper(most imp) 
        self.text_data = ''.join(filter(str.isalnum, self.text_data)).upper()
    

    def Save2CSV(self, filename=None, dir_name='output'):
        """
        Saves register df to csv
        Args:
            filename [str] - filename to save csv without extension. default original textfile name
            dir_name [str] - output folder to save csv in. default 'output'
        """
        if filename is not None:
            if not isinstance(filename, str):
                filename = str(filename)
            filename += '_' + self.count_method + '.csv'
        else:
            filename = self.filename + '_' + self.count_method + '.csv'

        output_directory = os.path.join(self.textFilesDirectory, dir_name)
        output_file_path = os.path.join(output_directory, filename)
     
        if not os.path.exists(output_directory):
            os.mkdir(output_directory)

        self.register.to_csv(output_file_path, index = False, header=True)


    def FormatRegister(self):
        """
        extract the compiled register.
        counts formatted as integer.
        NaN values replaced by 0's.
        Obtain metrics (min, max, std, var, average, avg %, mae)
        """
        self.register = self.register.fillna(0)
        for column in self.register.columns[1:]:
            self.register[column] = self.register[column].astype('int64')

        # get metrics
        Min = self.register.iloc[:,1:].min(axis=1, skipna=True).copy()
        Max = self.register.iloc[:,1:].max(axis=1, skipna=True).copy()
        Average = self.register.iloc[:,1:].mean(axis=1, skipna=True).apply(np.ceil).astype(int).copy()
        Std = self.register.iloc[:,1:].std(axis=1, skipna=True).round(4).copy()
        Var = self.register.iloc[:,1:].var(axis=1, skipna=True).round(4).copy()
        MAE = self.register.iloc[:,1:].var(axis=1, skipna=True).round(4).copy()
        
        # change average from float to int rounded up
        self.register['Minimum'] = Min
        self.register['Maximum'] = Max
        self.register['Average'] = Average
        self.register['STD'] = Std
        self.register['Variance'] = Var

        # computing average percentage
        avg_sum = np.sum(self.register['Average'], axis=None)
        avg_percent = (self.register['Average'].copy()/avg_sum)*100
        self.register['Avg %'] = avg_percent.round(4) # round to 2dps
        self.register.sort_values(by='Avg %', ascending=False, inplace=True)

        # computing the mean absolute error sum(|e_i|)/n
        self.register['Mean Absolute Error'] = self.register.iloc[:,1: self.n_experiments+1].copy().apply(lambda x: abs(x-Average[x.index])).mean(axis=1)


    def AddCounts2Register(self, exp_number, rows):
        """
        Implementation:
            Adds letter:frequency to register. If letter already exists, just add frequency to current exp_number. 
        Args:
            exp_number [int] - current experiment number
        Updates:
            register [int] - add letter frequency/count to register
        """
        for letter in rows:
            if letter not in self.register.values:
                self.register = self.register.append({'Letter':letter, f'Exp {exp_number}':int(rows[letter])}, ignore_index = True)
            else:
                self.register.loc[self.register['Letter']==letter, f'Exp {exp_number}'] = rows[letter]


    def FixedProbCounter(self, n_experiments=1):
        """
        Finds the actual number of distint letters in a literary work, with a probability of 1/2 of counting a new occurence.
        """
        self.n_experiments = n_experiments
        self.register = pd.DataFrame(columns=['Letter'])
        self.count_method = 'FixedProbCounter'

        for exp_number in range(1, n_experiments + 1):
            self.register[f'Exp {exp_number}'] = 0

            rows = dict()
            for letter in self.text_data:
                exp_prob = random.random()
                
                if exp_prob > self.prob:
                    if letter in rows:
                        rows[letter] += 1
                    else:
                        rows.update({letter : 1})
            
            # print('Experiment (', exp_number, ') for', self.count_method,'Done!')

            self.AddCounts2Register(exp_number, rows)
        self.FormatRegister()


    def DecreasingProbCounter(self, n_experiments=1):
        """
        counts new letter occurence with decreasing probability each time. 
        For instance, the 10'th occurence will have probability 1/sqrt(2)**10 probability of getting counted.
        This means the more frequent 
        """
        self.n_experiments = n_experiments
        self.register = pd.DataFrame(columns=['Letter'])
        self.count_method = 'DecreasingProbCounter'

        for exp_number in range(1, n_experiments + 1):
            self.register[f'Exp {exp_number}'] = 0

            rows = dict()
            for letter in self.text_data:
                
                exp_prob = random.random()
                prob = 1 - (np.sqrt(self.prob)**self.text_data.index(letter))

                if exp_prob > prob:
                    if letter in rows:
                        rows[letter] += 1
                    else:
                        rows.update({letter : 1})
            
            # print('Experiment (', exp_number, ') for', self.count_method,'Done!')

            self.AddCounts2Register(exp_number, rows)
        self.FormatRegister()
            

    def ExactCounter(self, n_experiments=1):
        """
        Get all letter, frequency for each experiment.
        Add letter, frequency to register
        """
        self.n_experiments = n_experiments
        self.register = pd.DataFrame(columns=['Letter'])
        self.count_method = 'ExactCounter'
        for exp_number in range(1, n_experiments + 1):
            self.register[f'Exp {exp_number}'] = 0

            rows = dict()
            for letter in self.text_data:
                if letter in rows:
                    rows[letter] += 1
                else:
                    rows.update({letter : 1})

            # print('Experiment (', exp_number, ') for', self.count_method,'Done!')

            self.AddCounts2Register(exp_number, rows)
        self.FormatRegister()



In [ ]:
literary_works = [
    'aMidsummerNightsDream-english',
    # 'aMidsummerNightsDream-french',
    # 'aMidsummerNightsDream-german',
    # 'hamlet-english',
    # 'hamlet-french',
    # 'hamlet-german',
    # 'juliusCaesar-english',
    # 'juliusCaesar-french',
    # 'juliusCaesar-german',
    # 'kingRichardIII-english',
    # 'kingRichardIII-french',
    # 'kingRichardIII-german',
    # 'macbeth-english',
    # 'macbeth-french',
    # 'macbeth-german',
    # 'merchantOfVenice-english',
    # 'merchantOfVenice-french',
    # 'merchantOfVenice-german',
    # 'othello-english',
    # 'othello-french',
    # 'othello-german',
    # 'romeoAndJuliet-english',
    # 'romeoAndJuliet-french',
    # 'romeoAndJuliet-german',
    # 'theTempest-english',
    # 'theTempest-french',
    # 'theTempest-german',
]

for literary_work in literary_works:
    AC = ApproximateCounter()
    AC.LoadTextFile(filename=literary_work)
    print('Working on',literary_work)
    
    print('started ExactCounter')
    start_exact = time.time()
    AC.ExactCounter(n_experiments=100)
    AC.Save2CSV()
    print('It took (', time.time() - start_exact, ') secs')

    print('started FixedProbCounter')
    start_fixed = time.time()
    AC.FixedProbCounter(n_experiments=100)
    AC.Save2CSV()
    print('It took (', time.time() - start_fixed, ') secs')

    print('started DecreasingProbCounter')
    start_decreased = time.time()
    AC.DecreasingProbCounter(n_experiments=100)
    AC.Save2CSV()
    print('It took (', time.time() - start_decreased, ') secs')

    print('-'*50)
    
print('Done!!!')

